## Checking Solaris Installation

In [1]:
import solaris as sol;
sol.__version__

/opt/conda/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework

'0.4.0'

## Listing the Data

In [2]:
from pathlib import Path

In [3]:
# intermediate directories
data_path = Path(Path.cwd()/'processedBuildingsLabels/data')
labels_path = Path(data_path/'labels')
rasters_vectors_path = Path(data_path/'rasters_vectors')

# files containing rasters & vectors data
b3 = Path(rasters_vectors_path/'3band')
b8 = Path(rasters_vectors_path/'8band')
geojson = Path(rasters_vectors_path/'geojson')

# directory containing training and preprocessing artifacts
artifacts_path = Path(Path.cwd()/'src/notebooks/artifacts')
df_path = Path(artifacts_path/'dataframes/df')
gdf_path = Path(artifacts_path/'dataframes/gdf')


models_path = Path(artifacts_path/'models')
xdxd_path = (Path(models_path/'xdxd'))

# directory containing yaml configuration files for model training
config_path = Path(Path.cwd()/'src/notebooks/config')
xdxd_model_config_path = Path(config_path/'xdxd_spacenet4.yml')

# directory containing inference csv
inference_in_path = Path(Path.cwd()/'inference_in')

# directory containing output csv upon inferencing
inference_out_path = Path(Path.cwd()/'inference_out')


Error: Pip module debugpy is required for debugging cells. You will need to install it to debug cells.

In [4]:
import pandas as pd
import geopandas as gpd

In [35]:
df = pd.read_pickle(df_path)
df.head()

,3band,8band,geojson,image_number,geo_extension,raster_extension_3,raster_extension_8,geoshape
0,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,1,geojson,tif,tif,"(0, 1)"
1,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,2,geojson,tif,tif,"(0, 1)"
2,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,3,geojson,tif,tif,"(0, 1)"
3,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,4,geojson,tif,tif,"(0, 1)"
4,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,5,geojson,tif,tif,"(0, 1)"


In [36]:
gdf = pd.read_pickle(gdf_path)
gdf.head()

,3band,8band,geojson,image_number,geo_extension,raster_extension_3,raster_extension_8,geoshape
0,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,12,geojson,tif,tif,"(5, 19)"
1,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,30,geojson,tif,tif,"(2, 19)"
2,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,44,geojson,tif,tif,"(1, 19)"
3,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,46,geojson,tif,tif,"(11, 19)"
4,processedBuildingLabels/data/rasters_vectors/3...,processedBuildingLabels/data/rasters_vectors/8...,processedBuildingLabels/data/rasters_vectors/g...,47,geojson,tif,tif,"(15, 19)"


## Creating the csv for desired inference data

In [55]:
inference_df = gdf['3band'][0:10]
# Change name of series for configuration purposes
inference_df.name = 'image'
# Save data as csv
inference_df.to_csv(Path(inference_in_path/'infer.csv'))
# Path to infer csv
infer_csv = Path(inference_in_path/'infer.csv')

## Getting pretrained model ready

In [7]:
config = sol.utils.config.parse(xdxd_model_config_path)
config

{'model_name': 'xdxd_spacenet4',
 'model_path': None,
 'train': False,
 'infer': True,
 'pretrained': True,
 'nn_framework': 'torch',
 'batch_size': 12,
 'data_specs': {'width': 512,
  'height': 512,
  'dtype': None,
  'image_type': 'zscore',
  'rescale': False,
  'rescale_minima': 'auto',
  'rescale_maxima': 'auto',
  'channels': 4,
  'label_type': 'mask',
  'is_categorical': False,
  'mask_channels': 1,
  'val_holdout_frac': 0.2,
  'data_workers': None},
 'training_data_csv': '/path/to/training_df.csv',
 'validation_data_csv': None,
 'inference_data_csv': '/path/to/test_df.csv',
 'training_augmentation': {'augmentations': {'DropChannel': {'idx': 3,
    'axis': 2},
   'HorizontalFlip': {'p': 0.5},
   'RandomRotate90': {'p': 0.5},
   'RandomCrop': {'height': 512, 'width': 512, 'p': 1.0},
   'Normalize': {'mean': [0.006479, 0.009328, 0.01123],
    'std': [0.004986, 0.004964, 0.00495],
    'max_pixel_value': 65535.0,
    'p': 1.0}},
  'p': 1.0,
  'shuffle': True},
 'validation_augmentati

### Configure Configuration file

#### Training Configuration

In [13]:
config['training']

{'epochs': 60,
 'steps_per_epoch': None,
 'optimizer': 'Adam',
 'lr': 0.0001,
 'opt_args': None,
 'loss': {'bcewithlogits': None, 'jaccard': None},
 'loss_weights': {'bcewithlogits': 10, 'jaccard': 2.5},
 'metrics': {'training': None, 'validation': None},
 'checkpoint_frequency': 10,
 'callbacks': {'model_checkpoint': {'filepath': 'xdxd_best.pth',
   'monitor': 'val_loss'}},
 'model_dest_path': 'xdxd.pth',
 'verbose': True}

In [19]:
config['training']['callbacks']['model_checkpoint']['filepath'] = str(xdxd_path)
config['training']['callbacks']['model_checkpoint']['filepath']

'/home/solaris/src/notebooks/artifacts/models/xdxd'

#### Inference Configuration

In [59]:
config['inference_data_csv'] =  'inference_in/'
config['inference_data_csv']

'inference_in/'

### Load Model

In [20]:
xdxd_inferer = sol.nets.infer.Inferer(config)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



/opt/conda/envs/solaris/lib/python3.7/site-packages/solaris/nets/model_io.py:35: UserWarning: The model weights file /opt/conda/envs/solaris/lib/python3.7/site-packages/solaris/nets/weights/xdxd_spacenet4_solaris_weights.pth was not found. Attempting to download from the SpaceNet repository.
  warn(f'The model weights file {model_path} was not found.'


solaris.nets.infer.Inferer

### Run inference

In [60]:
inf_df = sol.nets.infer.get_infer_df(config)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [68]:
xdxd_inferer(inference_df.astype(str).to_frame())


IndexError: index 3 is out of bounds for axis 2 with size 3

In [66]:
for idx, im_path in enumerate(inference_df):
    print(idx)
    print(type(im_path))

0
<class 'pathlib.PosixPath'>
1
<class 'pathlib.PosixPath'>
2
<class 'pathlib.PosixPath'>
3
<class 'pathlib.PosixPath'>
4
<class 'pathlib.PosixPath'>
5
<class 'pathlib.PosixPath'>
6
<class 'pathlib.PosixPath'>
7
<class 'pathlib.PosixPath'>
8
<class 'pathlib.PosixPath'>
9
<class 'pathlib.PosixPath'>


In [67]:
inference_df.astype(str)

0    processedBuildingLabels/data/rasters_vectors/3...
1    processedBuildingLabels/data/rasters_vectors/3...
2    processedBuildingLabels/data/rasters_vectors/3...
3    processedBuildingLabels/data/rasters_vectors/3...
4    processedBuildingLabels/data/rasters_vectors/3...
5    processedBuildingLabels/data/rasters_vectors/3...
6    processedBuildingLabels/data/rasters_vectors/3...
7    processedBuildingLabels/data/rasters_vectors/3...
8    processedBuildingLabels/data/rasters_vectors/3...
9    processedBuildingLabels/data/rasters_vectors/3...
Name: image, dtype: object